In [1]:
import pandas as pd
import numpy as np
import ast
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 데이터 확인

In [2]:
port = pd.read_csv("C:/Users/minji/전민정ㅎ/실전 프로젝트/archive (8)/portfolio.csv",index_col=0)
port.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [3]:
profile = pd.read_csv("C:/Users/minji/전민정ㅎ/실전 프로젝트/archive (8)/profile.csv",index_col=0)
profile.head()

,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [4]:
trans = pd.read_csv("C:/Users/minji/전민정ㅎ/실전 프로젝트/archive (8)/transcript.csv",index_col=0)
trans.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [5]:
port.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 560.0+ bytes


In [6]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 796.9+ KB


In [7]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
Index: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 11.7+ MB


In [8]:
profile.isnull().sum()

gender              2175
age                    0
id                     0
became_member_on       0
income              2175
dtype: int64

## 

In [9]:
trans['value'] = trans['value'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
trans['amount'] = trans['value'].apply(lambda x: x.get('amount') if isinstance(x, dict) else None)
trans['amount'] = trans['amount'].astype(float)
trans['offer_id'] = trans['value'].apply(lambda x: x.get('offer id') or x.get('offer_id') if isinstance(x, dict) else None)

In [10]:
port['channels'] = port['channels'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
port['web'] = port['channels'].apply(lambda x: 1 if isinstance(x, list) and 'web' in x else 0)
port['email'] = port['channels'].apply(lambda x: 1 if isinstance(x, list) and 'email' in x else 0)
port['mobile'] = port['channels'].apply(lambda x: 1 if isinstance(x, list) and 'mobile' in x else 0)
port['social'] = port['channels'].apply(lambda x: 1 if isinstance(x, list) and 'social' in x else 0)

In [11]:
income_median = profile['income'].median()
profile['income'].fillna(income_median, inplace=True)
profile['income']

0         64000.0
1        112000.0
2         64000.0
3        100000.0
4         64000.0
           ...   
16995     54000.0
16996     72000.0
16997     73000.0
16998     50000.0
16999     82000.0
Name: income, Length: 17000, dtype: float64

In [12]:
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')
profile['became_member_on'] = profile['became_member_on'].dt.date

In [13]:
import ast

def safe_eval(x):
    if isinstance(x, str):  # 문자열이면 변환
        return ast.literal_eval(x)
    return x  # 이미 리스트면 그대로 반환

port['channels'] = port['channels'].apply(safe_eval)


def make_offer_code(row):
    # offer_type 줄임말
    type_map = {
        'bogo': 'BOGO',
        'discount': 'DISC',
        'informational': 'INFO'
    }
    offer_type_short = type_map.get(row['offer_type'], row['offer_type'].upper())
    
    # 채널 약어 변환
    channel_map = {
        'email': 'E',
        'mobile': 'M',
        'social': 'S',
        'web': 'W'
    }
    channel_short = "".join([channel_map.get(ch, ch[0].upper()) for ch in row['channels']])
    
    # 최종 코드
    #return f"{offer_type_short}_R{row['reward']}_{row['duration']}D_Lv{row['difficulty']}"
    return f"{offer_type_short}_R{row['reward']}_{channel_short}_{row['duration']}D_Lv{row['difficulty']}"

# 새 컬럼 생성
port['offer_code'] = port.apply(make_offer_code, axis=1)
# 확인
port[['id', 'offer_code']].head(10)

,id,offer_code
0,ae264e3637204a6fb9bb56bc8210ddfd,BOGO_R10_EMS_7D_Lv10
1,4d5c57ea9a6940dd891ad53e9dbe8da0,BOGO_R10_WEMS_5D_Lv10
2,3f207df678b143eea3cee63160fa8bed,INFO_R0_WEM_4D_Lv0
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,BOGO_R5_WEM_7D_Lv5
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,DISC_R5_WE_10D_Lv20
5,2298d6c36e964ae4a3e7e9706d1fb8c2,DISC_R3_WEMS_7D_Lv7
6,fafdcd668e3743c1bb461111dcafc2a4,DISC_R2_WEMS_10D_Lv10
7,5a8bc65990b245e5a138643cd4eb9837,INFO_R0_EMS_3D_Lv0
8,f19421c1d4aa40978ebb69ca19b0e20d,BOGO_R5_WEMS_5D_Lv5
9,2906b810c7d4411798c6938adc9daaa5,DISC_R2_WEM_7D_Lv10


In [15]:
# 1. portfolio에서 id → offer_code 매핑 만들기
offer_map = dict(zip(port['id'], port['offer_code']))
# 2. transaction에 offer_code 붙이기
trans['offer_code'] = trans['offer_id'].map(offer_map)

In [18]:
trans.drop(columns=['value'], axis=1, inplace=True)
port.drop(columns=['channels'], axis=1, inplace=True)

In [19]:
trans.to_csv('transcript.csv', index=False)
port.to_csv('portfolio.csv', index=False)
profile.to_csv('profile.csv', index=False)